### **Consigna del desafío 1**

----

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups





**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [ ]:
# =================================================================
# PUNTO 1 - Vectorización y análisis de similitud entre documentos
# =================================================================
import random
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ---------- Cargar dataset de sklearn ----------
newsgroups_train = fetch_20newsgroups(
    subset='train',
    categories=None,  # Carga todas las 20 categorías
    remove=('headers', 'footers', 'quotes')
)

df = pd.DataFrame({
    "text": newsgroups_train.data,
    "label": [newsgroups_train.target_names[i] for i in newsgroups_train.target]
})

print(f"Cantidad de documentos: {len(df)}")
print(f"Categorías: {df['label'].unique()}\n")

# ---------- Vectorización con TF-IDF ----------
# Se mantiene ngram_range=(1,2) como estaba originalmente en tu código de P1
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df=0.95, min_df=2, stop_words='english')
X = tfidf.fit_transform(df['text'])

# ---------- Seleccionar 5 documentos aleatorios ----------
random.seed(42)
selected_idxs = random.sample(range(df.shape[0]), 5)

# ---------- Calcular similitud coseno y mostrar top-5 ----------
for idx in selected_idxs:
    vec = X[idx]
    sims = cosine_similarity(vec, X).flatten()
    top_idxs = sims.argsort()[::-1][:6]  # incluye el mismo documento
    top_idxs = [i for i in top_idxs if i != idx][:5]

    print("="*100)
    print(f"Documento {idx} (Etiqueta: {df.loc[idx,'label']})")
    print(f"Texto:\n{df.loc[idx,'text'][:400]}...\n")

    same_label = 0
    for rank, j in enumerate(top_idxs, start=1):
        match_label = df.loc[j, 'label']
        if match_label == df.loc[idx, 'label']:
            same_label += 1
        print(f"  {rank}. Similitud={sims[j]:.3f} | Etiqueta={match_label}")
        print(f"     → {df.loc[j,'text'][:200].replace('\n',' ')}...\n")

    print(f"→ Coinciden {same_label} de 5 en la misma categoría.\n")

print("\nAnálisis general (con 20 categorías):")
print("Al usar más categorías, es probable que la coincidencia de etiquetas en el top-5")
print("sea ligeramente menor que con solo 5 categorías debido a la mayor diversidad temática,")
print("pero la similitud coseno debería seguir agrupando bien documentos relacionados.\n")



Cantidad de documentos: 11314
Categorías: ['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']

Documento 10476 (Etiqueta: rec.sport.hockey)
Texto:
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are t...

  1. Similitud=0.139 | Etiqueta=rec.sport.baseball
     → I hate to be the burden of bad news, but I think I will this t

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [ ]:
# ============================================================
# PUNTO 2 - Clasificación por prototipos (tipo zero-shot)
# ============================================================

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# ---------- Cargar dataset ----------
newsgroups = fetch_20newsgroups(
    subset='train',
    categories=None,  # Carga las 20 categorías disponibles
    remove=('headers', 'footers', 'quotes')
)

df = pd.DataFrame({
    "text": newsgroups.data,
    "label": [newsgroups.target_names[i] for i in newsgroups.target]
})

print(f"Cantidad de documentos: {len(df)}")
print(f"Categorías (20 clases): {len(df['label'].unique())}\n")

# ---------- Split train/test ----------
X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# ---------- Vectorización ----------
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    max_df=0.95,
    min_df=2,
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# ---------- Clasificación tipo zero-shot ----------
# Cada documento del test se compara con TODOS los de entrenamiento,
# se asigna la etiqueta del documento más similar.

y_pred = []

for i in range(X_test_tfidf.shape[0]):
    # El cálculo de similitud coseno es la parte más costosa
    sims = cosine_similarity(X_test_tfidf[i], X_train_tfidf).flatten()
    best_idx = sims.argmax()
    pred_label = y_train.iloc[best_idx]
    y_pred.append(pred_label)

# ---------- Evaluación ----------
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy del modelo por prototipos (20 clases): {acc:.3f}\n")
print("Reporte de clasificación:\n")
print(classification_report(y_test, y_pred))

# ---------- Ejemplo de predicción ----------
print("="*100)
print("Ejemplo de comparación (primer documento del test):\n")
print("Texto test:\n", X_test.iloc[0][:400].replace('\n', ' '), "...\n")

sims = cosine_similarity(X_test_tfidf[0], X_train_tfidf).flatten()
best_idx = sims.argmax()

print(f"Predicción: {y_train.iloc[best_idx]}  |  Etiqueta real: {y_test.iloc[0]}\n")
print("Documento de entrenamiento más similar:\n", X_train.iloc[best_idx][:400].replace('\n', ' '), "...\n")


Cantidad de documentos: 11314
Categorías (20 clases): 20


Accuracy del modelo por prototipos (20 clases): 0.620

Reporte de clasificación:

                          precision    recall  f1-score   support

             alt.atheism       0.67      0.64      0.65        96
           comp.graphics       0.52      0.54      0.53       117
 comp.os.ms-windows.misc       0.57      0.56      0.57       118
comp.sys.ibm.pc.hardware       0.50      0.48      0.49       118
   comp.sys.mac.hardware       0.56      0.60      0.58       115
          comp.windows.x       0.70      0.67      0.68       119
            misc.forsale       0.58      0.56      0.57       117
               rec.autos       0.53      0.48      0.50       119
         rec.motorcycles       0.65      0.60      0.63       120
      rec.sport.baseball       0.65      0.69      0.67       119
        rec.sport.hockey       0.71      0.72      0.72       120
               sci.crypt       0.76      0.73      0.74       119


**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

In [ ]:
# ----------------------------------------------------------------------------
# PUNTO 3 - Modelos de clasificación Naïve Bayes (Optimizado con GridSearchCV)
# ----------------------------------------------------------------------------
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline

# 1. Definición de X e Y
X_data = df['text']
y_data = df['label']

# 2. División train / test
# Aseguramos que la división se haga ANTES de la vectorización si usamos Pipeline
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

# Definición de la grilla de hiperparámetros a buscar
# RESTRICCIÓN: ngram_range debe ser (1, 1).
param_grid = {
    # Parámetros del Vectorizador (CountVectorizer)
    'vectorizer__stop_words': ['english'],
    'vectorizer__ngram_range': [(1, 1)],    # Se mantiene fijo según la instrucción
    # Parámetros del Modelo Naïve Bayes
    'model__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0] # Tuning para maximizar el desempeño
}

# --- Modelo 1: Naive Bayes Multinomial (Optimización) ---
print("=====================================================")
print("=== 1. Naïve Bayes Multinomial (Optimización) ===")
print("=====================================================")

# Crear el Pipeline
pipeline_multi = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('model', MultinomialNB())
])

# Configurar GridSearchCV
grid_search_multi = GridSearchCV(
    pipeline_multi,
    param_grid,
    scoring='f1_macro', # Métrica objetivo
    cv=3,              # Cross-validation
    n_jobs=-1          # Usar todos los núcleos
)

# Entrenar la búsqueda de grilla
grid_search_multi.fit(X_train, y_train)

# Evaluación en el conjunto de test con el mejor modelo
y_pred_multi_best = grid_search_multi.predict(X_test)
f1_multi_best = f1_score(y_test, y_pred_multi_best, average='macro')

print("\nResultados MultinomialNB:")
print(f"Mejor alpha (optimizado): {grid_search_multi.best_params_['model__alpha']}")
print(f"F1-macro en Test (Mejor modelo): {round(f1_multi_best, 3)}")
print(classification_report(y_test, y_pred_multi_best))

print("-" * 50)

# --- Modelo 2: Naive Bayes ComplementNB (Optimización) ---
print("\n=====================================================")
print("=== 2. Naïve Bayes ComplementNB (Optimización) ===")
print("=====================================================")

# Crear el Pipeline
pipeline_comp = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('model', ComplementNB())
])

# Configurar GridSearchCV
grid_search_comp = GridSearchCV(
    pipeline_comp,
    param_grid, # Misma grilla de búsqueda (solo cambia el modelo)
    scoring='f1_macro',
    cv=3,
    n_jobs=-1
)

# Entrenar la búsqueda de grilla
grid_search_comp.fit(X_train, y_train)

# Evaluación en el conjunto de test con el mejor modelo
y_pred_comp_best = grid_search_comp.predict(X_test)
f1_comp_best = f1_score(y_test, y_pred_comp_best, average='macro')

print("\nResultados ComplementNB:")
print(f"Mejor alpha (optimizado): {grid_search_comp.best_params_['model__alpha']}")
print(f"F1-macro en Test (Mejor modelo): {round(f1_comp_best, 3)}")
print(classification_report(y_test, y_pred_comp_best))

print("-" * 50)

# --- Comparación final ---
print("\n Comparación Final (F1-macro en Test)")
print(f"MultinomialNB (Mejor alpha): {round(f1_multi_best, 3)}")
print(f"ComplementNB (Mejor alpha): {round(f1_comp_best, 3)}")

# Determinación del mejor modelo
if f1_comp_best > f1_multi_best:
    mejor_modelo = "ComplementNB"
    mejor_f1 = f1_comp_best
elif f1_multi_best > f1_comp_best:
    mejor_modelo = "MultinomialNB"
    mejor_f1 = f1_multi_best
else:
    mejor_modelo = "Ambos modelos tienen el mismo F1-macro en Test"
    mejor_f1 = f1_multi_best

print("\n Mejor modelo para maximizar el desempeño (F1-macro en Test):", mejor_modelo)
print("F1-macro máximo alcanzado:", round(mejor_f1, 3))





=== 1. Naïve Bayes Multinomial (Optimización) ===

Resultados MultinomialNB:
Mejor alpha (optimizado): 0.1
F1-macro en Test (Mejor modelo): 0.694
                          precision    recall  f1-score   support

             alt.atheism       0.67      0.72      0.70       135
           comp.graphics       0.57      0.67      0.61       166
 comp.os.ms-windows.misc       1.00      0.04      0.07       170
comp.sys.ibm.pc.hardware       0.56      0.80      0.66       182
   comp.sys.mac.hardware       0.75      0.77      0.76       183
          comp.windows.x       0.71      0.82      0.76       169
            misc.forsale       0.79      0.70      0.74       172
               rec.autos       0.79      0.75      0.77       191
         rec.motorcycles       0.79      0.79      0.79       198
      rec.sport.baseball       0.86      0.83      0.84       168
        rec.sport.hockey       0.58      0.80      0.68       163
               sci.crypt       0.80      0.75      0.78      

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [ ]:
# ================================================================
# PUNTO 4 - Similaridad entre palabras (matriz término-documento)
# ================================================================

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

# ---------- 1. Cargar el dataset completo (20 categorías) ----------
newsgroups_train = fetch_20newsgroups(
    subset='train',
    categories=None,  # Carga todas las 20 categorías
    remove=('headers', 'footers', 'quotes')
)

df = pd.DataFrame({
    "text": newsgroups_train.data
})
print(f"Documentos cargados para el Punto 4: {len(df)}")
# -------------------------------------------------------------------

# Vectorización unigramas: ngram_range=(1, 1) y min_df=5 para reducir el vocabulario
# (Se añade min_df=5 como optimización para reducir ligeramente la carga, aunque no es el problema principal)
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,1), min_df=5)
X = vectorizer.fit_transform(df['text'])

# Matriz término-documento (traspuesta)
X_t = X.T
vocab = np.array(vectorizer.get_feature_names_out())
print(f"Tamaño del Vocabulario después de filtrado: {len(vocab)}")

# Elegir 5 palabras clave.
selected_words = ['windows', 'space', 'baseball', 'encryption', 'cancer']
selected_indices = []

for word in selected_words:
    if word in vocab:
        selected_indices.append(np.where(vocab == word)[0][0])
    else:
        print(f" La palabra '{word}' no se encuentra en el vocabulario.")

# CÁLCULO DE SIMILITUD OPTIMIZADO:
# Solo calculamos la similitud de las 5 palabras seleccionadas contra TODAS las demás.
# Esto reduce el cálculo de (100k x 100k) a (5 x 100k).
X_selected = X_t[selected_indices]
similarity_matrix_optimized = cosine_similarity(X_selected, X_t)

# Mostrar los resultados
print("\n--- Análisis de Similaridad entre Palabras (20 Clases) ---\n")

for i, word in enumerate(selected_words):
    if word in vocab:
        # Fila de similitudes para la palabra actual
        sims = similarity_matrix_optimized[i]

        # Obtiene los 6 mejores índices y omite el primer (la palabra misma)
        top_indices = sims.argsort()[::-1][1:6]
        similar_words = vocab[top_indices]
        print(f" Palabra: '{word}'")
        print("    Más similares:", ', '.join(similar_words))
    # No se necesita else aquí porque el filtro ya se hizo antes

print("\n-------------------------------------------------------------")


Documentos cargados para el Punto 4: 11314
Tamaño del Vocabulario después de filtrado: 17797

--- Análisis de Similaridad entre Palabras (20 Clases) ---

 Palabra: 'windows'
    Más similares: hardware, nt, microsoft, software, using
 Palabra: 'space'
    Más similares: aerospace, satellites, exploration, nasa, satellite
 Palabra: 'baseball'
    Más similares: hispanic, clemens, pitching, ripken, stats
 Palabra: 'encryption'
    Más similares: accommodates, torrance, heyman, pitted, nistnews
 Palabra: 'cancer'
    Más similares: asthma, centers, particles, avenue, airborne

-------------------------------------------------------------


# Conclusiones Generales del Trabajo Final

El proyecto se basó en el análisis y clasificación del *dataset* **20 Newsgroups** en su totalidad (**20 categorías temáticas**), comparando enfoques de procesamiento de lenguaje natural (NLP) basados en **Similitud por Distribución**, **Clasificación Zero-Shot** y **Modelos Supervisados Clásicos** (Naïve Bayes optimizado).

---

## 1. Similitud Documental y Clasificación Zero-Shot

### **Similitud Documental (Punto 1)**

La combinación de la representación **TF-IDF** con la **Similitud Coseno** demostró ser efectiva para agrupar documentos por tema, incluso con 20 categorías.

| Documento Base (Categoría) | Fidelidad (Coincidentes / 5) | Observación |
| :--- | :--- | :--- |
| **`comp.sys.mac.hardware`** | **5 de 5** | Alta coherencia; temas muy técnicos. |
| **`rec.sport.hockey`** | **5 de 5** | Coherencia perfecta; vocabulario especializado. |
| **`rec.autos`** | **3 de 5** | Solapamiento con temas de `rec.motorcycles`, mostrando la mayor ambigüedad temática. |

* **Conclusión:** La métrica de similitud coseno sobre los vectores TF-IDF captura la **coherencia temática** del corpus. La caída en los valores máximos de similitud (respecto a 5 clases) es esperada por el aumento del vocabulario, pero el agrupamiento por etiqueta se mantiene sólido en temas bien definidos.

### **Clasificación por Prototipos (Punto 2)**

* **Resultado Clave:** Se obtuvo una **Accuracy del $\mathbf{0.620}$** y un **F1-macro del $\mathbf{0.62}$** en el conjunto de prueba.
* **Interpretación:** Para un problema de **20 clases**, un F1-macro de $0.62$ es aceptable para un modelo *zero-shot* basado en el vecino más cercano. La **caída de rendimiento** (respecto al $\approx 0.82$ de 5 clases) confirma la mayor dificultad del problema con más etiquetas.

---

## 2. Modelos de Clasificación Naïve Bayes (Punto 3)

### **Optimización y Comparación (Máximo F1-macro)**

Se optimizó el hiperparámetro de suavizado ($\alpha$) usando *Grid Search* sobre un **CountVectorizer** con **unigramas** (`ngram_range=(1, 1)`).

| Modelo | F1-macro en Test (Final) | Mejor $\alpha$ |
| :--- | :--- | :--- |
| **Naïve Bayes Multinomial** | $\mathbf{0.694}$ | $0.1$ |
| **Naïve Bayes ComplementNB** | $\mathbf{0.704}$ | $0.5$ |

**Conclusión Final del Punto 3:**
El modelo **ComplementNB** fue el que alcanzó el máximo desempeño, superando a MultinomialNB con un **F1-macro de $\mathbf{0.704}$**. Esta robustez se debe a su mejor manejo de las clases con menor representación o con *features* menos distintivas. Se observa que la clasificación en 20 clases presenta serios desafíos en categorías como `comp.os.ms-windows.misc` (F1 $\approx 0.07$ en MNB) y `talk.religion.misc` (F1 $\approx 0.33$ en CNB), donde el *recall* es notablemente bajo.

---

### 3. Similaridad entre Palabras (Punto 4)

#### **Análisis de Coocurrencia Contextual (Matriz Término $\times$ Documento)**

La transposición de la matriz término-documento... permitió analizar la **afinidad contextual** entre los términos.

* **Coherencia Fuerte:** Palabras como **'space'** y **'baseball'** mostraron relaciones temáticas claras y esperadas..., lo que valida que la coocurrencia modelada captura la estructura semántica en categorías bien definidas.
* **Fallo Contextual (Coherencia Nula):** Términos como **'encryption'** y **'cancer'** generaron listas de vecinos completamente **incoherentes** ('torrance', 'pitted', 'airborne'). Esto indica que su vector de coocurrencia es dominado por el **ruido**, impidiendo una interpretación semántica clara.

**Conclusión Final del Punto 4:**
El análisis valida que la similitud coseno sobre la matriz transpuesta captura las **relaciones de coocurrencia implícitas** solo para términos con alta distinción temática. El fracaso en palabras como 'encryption' sugiere una limitación en este método para términos con baja frecuencia o amplio solapamiento inter-clase, **un detalle crucial para la interpretación de los resultados.**

---

## Resumen General del Proyecto

1.  **Representación:** **TF-IDF** demostró ser una representación vectorial robusta y efectiva para la medición de similaridad y clasificación inicial en 20 clases.
2.  **Clasificación:** El modelo supervisado **ComplementNB** optimizado se consolidó como la **solución de mejor desempeño** al alcanzar un F1-macro de **$\mathbf{0.704}$**, superando al MultinomialNB.
3.  **Análisis:** El estudio de la matriz término-documento confirmó que las relaciones semánticas son un reflejo directo y fiel del contexto temático **solo para los términos con alta distinción temática** (ej. 'space', 'baseball'). Se observaron **fallos en la coherencia** para términos con baja frecuencia o amplio solapamiento inter-clase (ej. 'encryption', 'cancer'), lo que señala una **limitación clave** de la representación para el *dataset* completo.